In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver import Chrome as wdChrome, Firefox as wdFirefox, Edge as wdEdge
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
# from selenium.webdriver.edge.service import Service as EdgeService
# from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions as selExcep 

from webdriver_manager.chrome import ChromeDriverManager
# from webdriver_manager.firefox import GeckoDriverManager
# from webdriver_manager.microsoft import EdgeChromiumDriverManager


import pandas as pd
import numpy as np
import time

In [4]:
home_url = 'https://www.google.com/maps/place/Lahore,+Punjab,+Pakistan/@74.1943055,31.4831569,11z/data=!3m1!4b1!4m5!3m4!1s0x39190483e58107d9:0xc23abe6ccc7e2462!8m2!3d31.5203696!4d74.3587473'
driver_type = "chrome"
service = ChromeService(executable_path = ChromeDriverManager().install())
driver = wdChrome(service= service)
driver.get(home_url)
action = ActionChains(driver)
time.sleep(1)

KeyboardInterrupt: 

In [14]:
MAP_SEARCH_XPATH = '//*[@id="searchboxinput"]'
LOCATION_BOX_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]'
LOC_BOX_CLASS = "hfpxzc"

In [15]:
LOC_NAME_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1'
LOC_RATING_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[1]/span/span[1]'
LOC_RATING_COUNT_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/button'

In [16]:
def search_map(query: str) -> None:
    map_searchbar = driver.find_element(By.XPATH, MAP_SEARCH_XPATH)
    map_searchbar.clear()
    map_searchbar.send_keys(query)
    map_searchbar.send_keys(Keys.RETURN)

### Collecting hrefs from all locations in a radius

need to add functionality to search all radius in a lat-long data grid

In [17]:
## Covers all of lahore

start_lat= 31.283179
start_long= 74.080275

stop_lat = 31.671291
stop_long = 74.551132

lat_dist = np.arange(start=start_lat, stop= stop_lat, step= 0.1, dtype= "float32")
long_dist = np.arange(start=start_long, stop= stop_long, step= 0.1, dtype= "float32")

geo_loc_grid = np.transpose([np.tile(lat_dist, len(long_dist)), np.repeat(long_dist, len(lat_dist))])

In [18]:
count = 0
for geo_loc in geo_loc_grid:
    print(geo_loc)
    count += 1
count

[31.283178 74.08028 ]
[31.383179 74.08028 ]
[31.48318 74.08028]
[31.58318 74.08028]
[31.283178 74.180275]
[31.383179 74.180275]
[31.48318  74.180275]
[31.58318  74.180275]
[31.283178 74.28027 ]
[31.383179 74.28027 ]
[31.48318 74.28027]
[31.58318 74.28027]
[31.283178 74.38027 ]
[31.383179 74.38027 ]
[31.48318 74.38027]
[31.58318 74.38027]
[31.283178 74.48027 ]
[31.383179 74.48027 ]
[31.48318 74.48027]
[31.58318 74.48027]


20

In [19]:
print(lat_dist.size)
print(lat_dist)
print()

print(long_dist.size)
print(long_dist)
print()

print(lat_dist.size * long_dist.size)

4
[31.283178 31.383179 31.48318  31.58318 ]

5
[74.08028  74.180275 74.28027  74.38027  74.48027 ]

20


In [20]:
search_map("resteraunts")

In [21]:
# loc_div = driver.find_element(By.XPATH, LOCATION_BOX_XPATH)
# action.move_to_element(loc_div).perform()
# hrefs_set = set()


# last_href = None
# find_elem = False
# while not find_all_elem:
    
#     loc_boxes = driver.find_elements(By.CLASS_NAME, LOC_BOX_CLASS)
#     hrefs = [loc_box.get_attribute("href") for loc_box in loc_boxes]
#     hrefs_set.update(hrefs)
#     loc_div.send_keys(Keys.END)
#     time.sleep(2)
    
#     if(last_href == None):
#         last_href = hrefs[-1]
#         continue
    
#     if(last_href == hrefs[-1]):
#         find_all_elem = True
#     else:
#         last_href = hrefs[-1]

In [22]:
hrefs_set = set()


last_href = None
find_all_elem = False
curr_url = driver.current_url
template_url = curr_url.split("@")[0]

for loc in geo_loc_grid:
    lat = loc[0]
    long = loc[1]
    dynamic_url = f"{template_url}@{lat},{long},13z/data=!3m1!4b1"
    print(dynamic_url)
    driver.get(dynamic_url)
    time.sleep(1)
    while not find_all_elem:
        loc_div = driver.find_element(By.XPATH, LOCATION_BOX_XPATH)
        action.move_to_element(loc_div).perform()
        loc_boxes = driver.find_elements(By.CLASS_NAME, LOC_BOX_CLASS)
        hrefs = [loc_box.get_attribute("href") for loc_box in loc_boxes]
        hrefs_set.update(hrefs)
        loc_div.send_keys(Keys.END)
        time.sleep(2)
        
        if(last_href == None):
            last_href = hrefs[-1]
            continue
        
        if(last_href == hrefs[-1]):
            find_all_elem = True
        else:
            last_href = hrefs[-1]
    find_all_elem = False
    time.sleep(5)

https://www.google.com/maps/search/resteraunts/@31.283178329467773,74.08027648925781,13z/data=!3m1!4b1
https://www.google.com/maps/search/resteraunts/@31.3831787109375,74.08027648925781,13z/data=!3m1!4b1
https://www.google.com/maps/search/resteraunts/@31.483179092407227,74.08027648925781,13z/data=!3m1!4b1
https://www.google.com/maps/search/resteraunts/@31.583179473876953,74.08027648925781,13z/data=!3m1!4b1
https://www.google.com/maps/search/resteraunts/@31.283178329467773,74.1802749633789,13z/data=!3m1!4b1
https://www.google.com/maps/search/resteraunts/@31.3831787109375,74.1802749633789,13z/data=!3m1!4b1
https://www.google.com/maps/search/resteraunts/@31.483179092407227,74.1802749633789,13z/data=!3m1!4b1
https://www.google.com/maps/search/resteraunts/@31.583179473876953,74.1802749633789,13z/data=!3m1!4b1
https://www.google.com/maps/search/resteraunts/@31.283178329467773,74.2802734375,13z/data=!3m1!4b1
https://www.google.com/maps/search/resteraunts/@31.3831787109375,74.2802734375,13z/da

In [23]:
hrefs_set

{'https://www.google.com/maps/place/%D8%A7%D9%84%D9%85%D8%AF%DB%8C%D9%86%DB%81+%DA%A9%D8%A7%DA%A9%D8%A7%D8%AE%DB%8C%D9%84+%D9%81%DB%8C%D9%85%D9%84%DB%8C+%D8%B1%DB%8C%D8%B3%D9%B9%D9%88%D8%B1%D9%86%D9%B9%E2%80%AD/data=!4m7!3m6!1s0x3918ff3d37d2ba25:0xcefcc2962593422d!8m2!3d31.403337!4d74.1584129!16s%2Fg%2F11sxl08nwk!19sChIJJbrSNz3_GDkRLUKTJZbC_M4?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/%D8%A7%D9%85%DB%8C%D9%86+%D9%85%D8%B1%D8%BA+%DA%86%D9%86%DB%92+%D9%86%D8%A7%D9%86+%D8%B4%D8%A7%D9%BE+%D8%A7%D9%84%D9%85%D8%AF%DB%8C%D9%86%DB%81+%D8%B1%DB%8C%D8%B3%D9%B9%D9%88%D8%B1%DB%8C%D9%86%D9%B9%E2%80%AD/data=!4m7!3m6!1s0x3919a591ca6972bb:0x24753edeedc7fe33!8m2!3d31.2511973!4d74.4204402!16s%2Fg%2F11j4lf2xvp!19sChIJu3JpypGlGTkRM_7H7d4-dSQ?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/%D8%B3%D8%B1%D9%88%D8%A7%D9%86%DB%8C+%D8%B1%D8%AD%D9%85%DA%A9%D8%A7%D8%B1%DB%8C%DB%81+%DB%81%D9%88%D9%B9%D9%84%E2%80%AD/data=!4m7!3m6!1s0x3918572a17d60c9d:0x2c9eb95fa06ba190!8m2!3d31.329

# Extracting data from href

In [24]:
df = pd.DataFrame()

### getting data from each href

In [25]:
href_list = list()
loc_names = list()
loc_ratings = list()
geo_locs = list()


In [26]:
href_list = list(hrefs_set)
href_list.sort()
href_list

['https://www.google.com/maps/place/%D8%A7%D9%84%D9%85%D8%AF%DB%8C%D9%86%DB%81+%DA%A9%D8%A7%DA%A9%D8%A7%D8%AE%DB%8C%D9%84+%D9%81%DB%8C%D9%85%D9%84%DB%8C+%D8%B1%DB%8C%D8%B3%D9%B9%D9%88%D8%B1%D9%86%D9%B9%E2%80%AD/data=!4m7!3m6!1s0x3918ff3d37d2ba25:0xcefcc2962593422d!8m2!3d31.403337!4d74.1584129!16s%2Fg%2F11sxl08nwk!19sChIJJbrSNz3_GDkRLUKTJZbC_M4?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/%D8%A7%D9%85%DB%8C%D9%86+%D9%85%D8%B1%D8%BA+%DA%86%D9%86%DB%92+%D9%86%D8%A7%D9%86+%D8%B4%D8%A7%D9%BE+%D8%A7%D9%84%D9%85%D8%AF%DB%8C%D9%86%DB%81+%D8%B1%DB%8C%D8%B3%D9%B9%D9%88%D8%B1%DB%8C%D9%86%D9%B9%E2%80%AD/data=!4m7!3m6!1s0x3919a591ca6972bb:0x24753edeedc7fe33!8m2!3d31.2511973!4d74.4204402!16s%2Fg%2F11j4lf2xvp!19sChIJu3JpypGlGTkRM_7H7d4-dSQ?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/%D8%B3%D8%B1%D9%88%D8%A7%D9%86%DB%8C+%D8%B1%D8%AD%D9%85%DA%A9%D8%A7%D8%B1%DB%8C%DB%81+%DB%81%D9%88%D9%B9%D9%84%E2%80%AD/data=!4m7!3m6!1s0x3918572a17d60c9d:0x2c9eb95fa06ba190!8m2!3d31.329

### Getting infro from each href

In [27]:
for href in href_list:
    lat, long = href.split("!3d")[1].split("!4d")
    long = long.split("!")[0]
    lat_long = (lat, long)
    
    driver.get(href)
    time.sleep(0.5)
    
    name = driver.find_element(By.XPATH, LOC_NAME_XPATH).text
    
    try:
        rating = driver.find_element(By.XPATH, LOC_RATING_XPATH).text
    except selExcep.NoSuchElementException:
        rating = None
    
    
    loc_names.append(name)
    loc_ratings.append(rating)
    geo_locs.append(lat_long)

In [28]:
df["loc_name"] = loc_names
df["loc_rating"] = loc_ratings
df["geo_location"] = geo_locs
df["href"] = href_list
df

,loc_name,loc_rating,geo_location,href
0,المدینہ کاکاخیل فیملی ریسٹورنٹ,4.5,"(31.403337, 74.1584129)",https://www.google.com/maps/place/%D8%A7%D9%84...
1,امین مرغ چنے نان شاپ المدینہ ریسٹورینٹ,5.0,"(31.2511973, 74.4204402)",https://www.google.com/maps/place/%D8%A7%D9%85...
2,سروانی رحمکاریہ ہوٹل,3.4,"(31.3299377, 74.1096191)",https://www.google.com/maps/place/%D8%B3%D8%B1...
3,طارق ہوٹل والا,5.0,"(31.4562605, 74.1001349)",https://www.google.com/maps/place/%D8%B7%D8%A7...
4,غریب نواز ملک اینڈ فوڈ پوائنٹ,None,"(31.2490089, 74.4175116)",https://www.google.com/maps/place/%D8%BA%D8%B1...
...,...,...,...,...
606,"Zinger Burger, Shawarma",4.1,"(31.5267001, 74.2639156)",https://www.google.com/maps/place/Zinger+Burge...
607,Zubair Restaurant,None,"(31.4749337, 74.3057269)",https://www.google.com/maps/place/Zubair+Resta...
608,gala restaurant,None,"(31.264664, 74.1250576)",https://www.google.com/maps/place/gala+restaur...
609,k2,3.9,"(31.3720913, 74.4633786)",https://www.google.com/maps/place/k2/data=!4m7...


In [29]:
df.to_csv('prototype.csv')

In [30]:
test_href = hrefs_set.pop()
test_href

'https://www.google.com/maps/place/Spicy+Chilli+Restaurant/data=!4m7!3m6!1s0x3919adbbf9ec4681:0x8f8e29126999b630!8m2!3d31.2574132!4d74.2258892!16s%2Fg%2F11qpvvz9t8!19sChIJgUbs-butGTkRMLaZaRIpjo8?authuser=0&hl=en&rclk=1'

In [31]:
driver.get(test_href)

In [32]:
rating_count = driver.find_element(By.XPATH, LOC_RATING_COUNT_XPATH).text.split()[0]
rating_count

'4'